# Quantum Circuits in the Munich Quantum Toolkit

The central interface for working with quantum circuits in MQT is the `QuantumComputation` class. It represents quantum circuits as a sequential list of operations. Operations can be directly applied to the `QuantumComputation`: 

In [ ]:
from mqt.core import QuantumComputation, Control, ControlType

# Build a `QuantumComputation` representing a Bell-state preparation circuit.
nqubits = 2
qc = QuantumComputation(nqubits)

qc.h(0)  # Apply Hadamard gate on qubit 0
qc.x(1, Control(0))  # Apply a CNOT (controlled X-Gate) with control on qubit 0 and target on qubit 1

# Get Circuit in OpenQASM 2.0 format
print(qc.qasm_str())

It might seem odd at first that the control qubit has to be explicitely constructed as a `Control` object. But this syntax actually provides a lot of flexibility as every unitary gate can be declared as a controlled gate:

In [ ]:
nqubits = 2
qc = QuantumComputation(nqubits)

# Controlled Hadamard Gate
qc.h(1, Control(0))

# Negatively controlled S-gate: S-Gate on target is performed if control is in |0> state.
qc.s(1, Control(0, ControlType.Neg))

print(qc.qasm_str())

Providing a set of `Control` objects allows declaring any gate as a multi-controlled gate:

In [ ]:
nqubits = 3
qc = QuantumComputation(nqubits)

# Toffoli gate in mqt-core:
qc.x(2, {Control(0), Control(1)})

# Control type can be individually declared
qc.s(0, {Control(1, ControlType.Neg), Control(2, ControlType.Pos)})

print(qc.qasm_str())

## Layout Information

A `QuantumComputation` also contains information about the mapping of logical (or algorithmic) qubits to and from physical (or device) qubits. These are contained in the `initial_layout` and `output_permutation` members which are instances of the `Permutation` class. If no layout is given the trivial layout is assumed. 

When printing the OpenQASM 2.0 representation of the `QuantumComputation` the input and output permutations are given as comments in the first two lines of the QASM string. The format is:

`// i q_0, q_1, ..., q_n` ... logical qubit $q_i$ is mapped to physical qubit $i$.

`// o c_0, c_1, ..., c_n` ... classical bit $c_i$ stores measurement result of qubit $i$.


In [ ]:
from mqt.core import Permutation

nqubits = 3
qc = QuantumComputation(3)
qc.initial_layout[0] = 2
qc.initial_layout[1] = 1
qc.initial_layout[2] = 0

qc.output_permutation[2] = 0
qc.output_permutation[1] = 2
qc.output_permutation[0] = 1


print(qc.qasm_str())

The layout information can also be automatically determined from measurements.

In [ ]:
nqubits = 3
qc = QuantumComputation(nqubits)

qc.h(0)
qc.x(1)
qc.s(2)
qc.measure(0, 2)  # measure qubit 0 and store result in classical bit 2
qc.measure(1, 0)  # measure qubit 1 and store result in classical bit 0
qc.measure(2, 1)  # measure qubit 2 and store result in classical bit 1
qc.initialize_io_mapping()  # determine permutation from measurement

print(qc.qasm_str())

## Operations

The operations in a `QuantumComputation` object are of type `Operation`. Every type of operation in `mqt-core` is derived from this class. Operations can also be explicitely constructed. Each `Operation` has a type in the form of an `OpType`.

### `StandardOperation`
A `StandardOperation` is used to represent basic unitary gates. These can also be declared with arbitrary targets and controls.

In [ ]:
from mqt.core import StandardOperation, OpType
from math import pi

nqubits = 4

# u3 gate on qubit 0 in a 3-qubit circuit
u3_gate = StandardOperation(nqubits, target=0, params=[pi / 2, pi, -pi / 2], op_type=OpType.u3)

# controlled x-rotation
cxr = StandardOperation(nqubits, target=1, control=Control(0), params=[pi], op_type=OpType.rx)

# multi-target multi-controlled x-gate
mcx = StandardOperation(nqubits, targets=[0, 1], controls={Control(2), Control(3)}, op_type=OpType.x)

# add operations to a quantum computation
qc = QuantumComputation(nqubits)
qc.append_operation(u3_gate)
qc.append_operation(cxr)
qc.append_operation(mcx)

### `NonUnitaryOperation`

A `NonUnitaryOperation` is used to represent operations involving measurements, resets or barriers.

In [ ]:
from mqt.core import NonUnitaryOperation

nqubits = 2

qc.h(0)

# measure qubit 0 on classical bit 0
meas_0 = NonUnitaryOperation(nqubits, target=0, classic=0)

# reset all qubits
barrier = NonUnitaryOperation(nqubits, targets=[0, 1], op_type=OpType.reset)

qc.append_operation(meas_0)
qc.append_operation(barrier)

### `SymbolicOperation`

A `SymbolicOperation` can represent all gates of a `StandardOperation` but the gate parameters can be symbolic. Symbolic expressions are represented in MQT using the `Expression` type, which represent linear combinations of symbolic `Term` objects over some set of `Variable` objects.

In [ ]:
from mqt.core import SymbolicOperation, Expression, Term, Variable

nqubits = 1

x = Variable("x")
y = Variable("y")
sym = Expression([Term(2, x), Term(3, y)])
print(sym)

sym += 1
print(sym)

# Create symbolic gate
u1_symb = SymbolicOperation(nqubits, target=0, params=[sym], op_type=OpType.phase)

# Mixed symbolic and instantiated parameters
u2_symb = SymbolicOperation(nqubits, target=0, params=[sym, 2.0], op_type=OpType.u2)

### `CompoundOperation`

A `CompoundOperation` bundles multiple `Operation` objects together. 

In [ ]:
from mqt.core import CompoundOperation

nqubits = 2
comp_op = CompoundOperation(nqubits)

# create bell pair circuit
comp_op.append_operation(StandardOperation(nqubits, 0, op_type=OpType.h))
comp_op.append_operation(StandardOperation(nqubits, target=0, control=Control(1), op_type=OpType.x))

qc = QuantumComputation(nqubits)
qc.append_operation(comp_op)

print(qc.qasm_str())

## Interfacing with other SDKs

Since a `QuantumComputation` can be imported from and exported to a OpenQASM 2.0 string, any library that can work with OpenQASM 2.0 is easy to use in conjunction with the `QuantumComputation` class. 

Currently, `mqt-core` can import [Qiskit](https://qiskit.org/) [`QuantumCircuit`](https://qiskit.org/documentation/stubs/qiskit.circuit.QuantumCircuit.html) objects directly. This should be faster than exporting and importing QASM strings.

In [ ]:
from qiskit import QuantumCircuit
from mqt.core.qiskit import qiskit_to_mqt

# GHZ circuit in qiskit
qiskit_qc = QuantumCircuit(3)
qiskit_qc.h(0)
qiskit_qc.cx(0, 1)
qiskit_qc.cx(0, 2)

print(qiskit_qc.draw())

mqt_qc = qiskit_to_mqt(qiskit_qc)
print(mqt_qc.qasm_str())